In [ ]:
import sys
sys.path.append("../")
import import_ipynb
from config import dogs_vs_cats_config as config
from pyimage.preprocessing.preprocessors import ImageToArrayPreprocessor, SimplePreprocessor, PatchPreprocessor, MeanPreprocessor
from pyimage.callbacks import TrainingMonitor
from pyimage.io.hd5py import HDF5DatasetGenerator
from pyimage.nn.alexnet import AlexNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizer import Adam
import json
import os

In [ ]:
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
                        width_shift_range=0.2, height_shift_range=0.2,
                        shear_range=0.15, horizontal_flip=True,
                        fill_mode="nearest")

means = json.loads(open(config.DATASET_MEAN)).read()

sp = SimplePreprocessor(227, 227)
pp = PatchPreprocessor(227, 227)
mp = MeanPreprocessor(mean["R"], means["G"], means["B"])
iap = ImageToArrayPreprocessor()

In [ ]:
# create our training dataset generator. Here we supply the path to our training
# HDF5 file, indicating that we should use batch sizes of 128 images, data augmentation, and three
# pre-processors: patch, mean, and image to array, respectively.
trainGen = HDF5DatasetGenerator(config.TRAIN_HDF5, 128, aug=aug,
                               preprocessors=[pp, mp, iap], classes=2)

# for instantiating the testing generator. This time we’ll supply
# the path to the validation HDF5 file, use a batch size of 128, no data augmentation, and a simple
# pre-processor rather than a patch pre-processor (since data augmentation is not applied to validation
# data). 
valGen = HDF5DatasetGenerator(config.VAL_HDF5, 128,
                             preprocessors=[sp, mp, iap], classes=2)

In [ ]:
# init the optimizer
print("INFO compiling model")
opt = Adam(lr=1e-3)
model = AlexNet.build(width=227, height=227, depth=3, classes=2, reg=0.002)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

path = os.path.sep.join([config.OUTPUT_PATH, "{}.png".format(os.getpid())])

In [ ]:
H = model.fit_generator(
    trainGen.generator(),
    steps_per_epoch=trainGen.numImages//128,
    validation_data=valgen.generator(),
    validation_steps=valGen.numImages//128,
    epochs=75,
    max_queue_size=10,
    verbose=1
)

In [ ]:
print("INFO serializing model")
model.save(config.MODEL_PATH, overwrite=True)

trainGen.close()
valGen.close()

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 75), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 75), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 75), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, 75), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()